In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
def read_csv(policy_name, dir, is_SB, case):
    dataframes = []
    filenames = []

    for root, dirs, files in os.walk(dir):
        # Filter directories within the root directory
        dirs[:] = [d for d in dirs if d.startswith(policy_name)]
        # Only proceed if we're in a directory that matches the folder prefix
        if os.path.basename(root).startswith(policy_name):
            # Loop through each file in the directory
            for file in files:
                # Check if file starts with the prefix
                if file.startswith(case):
                    file_path = os.path.join(root, file)
                    try:
                        # Load the file into a DataFrame
                        df = pd.read_csv(file_path)
                        # Store the DataFrame in the dictionary using the path as a key
                        dataframes.append(df)
                        filenames.append(file)
                    except Exception as e:
                        print(f"Failed to read {file_path}: {e}")
    columns_sb = ['Bike Arrivals', 'Bike Starvations', 'Long Congestions']
    
    columns_ff = ['Escooter Arrivals', 'Escooter Starvations', 'Battery Starvations', 'Battery Violations']
    
    fix_dfs = []
    for df in dataframes:
        selected_columns = columns_sb if is_SB else columns_ff
        fix_dfs.append(df[selected_columns])
    return fix_dfs, filenames

In [3]:
def make_pie(column_means):
    # Calculate the percentage contribution of each column's mean
    total_means = column_means.sum()
    percentage_means = (column_means / total_means) * 100

    # Plotting the pie chart
    plt.figure(figsize=(8, 8))  # Adjust the figure size as necessary
    plt.pie(percentage_means, labels=percentage_means.index, autopct='%1.1f%%', startangle=90)
    
    plt.show()

### Rank the tests

In [6]:
dir = '/Users/isabellam/NTNU/H2023/Prosjektoppgave/fomo/policies/hlv_master/criticality_results/'
dataframes, filenames = read_csv('Base', dir, False, 'criticality')
filenames

['criticality_weights_o_Base.csv',
 'criticality_weights_n_Base.csv',
 'criticality_weights_e_Base.csv',
 'criticality_weights_d_Base.csv',
 'criticality_weights_aa_Base.csv',
 'criticality_weights_z_Base.csv',
 'criticality_weights_q_Base.csv',
 'criticality_weights_p_Base.csv',
 'criticality_weights_v_Base.csv',
 'criticality_weights_w_Base.csv',
 'criticality_weights_h_Base.csv',
 'criticality_weights_i_Base.csv',
 'criticality_weights_b_Base.csv',
 'criticality_weights_c_Base.csv',
 'criticality_weights_r_Base.csv',
 'criticality_weights_s_Base.csv',
 'criticality_weights_x_Base.csv',
 'criticality_weights_ab_Base.csv',
 'criticality_weights_y_Base.csv',
 'criticality_weights_ac_Base.csv',
 'criticality_weights_f_Base.csv',
 'criticality_weights_g_Base.csv',
 'criticality_weights_l_Base.csv',
 'criticality_weights_m_Base.csv',
 'criticality_weights_a_Base.csv',
 'criticality_weights_k_Base.csv',
 'criticality_weights_j_Base.csv',
 'criticality_weights_u_Base.csv',
 'criticality_wei

In [7]:
means = []
for i in range(len(dataframes)):
    dataframe = dataframes[i]
    filename = filenames[i]
    
    mean_series = dataframe.mean()
    mean_series['Filename'] = filename
    means.append(mean_series)

new_df = pd.DataFrame(means)
new_df['Sum'] = new_df.drop(columns=['Filename', 'Escooter Arrivals']).sum(axis=1)
new_df['Lost trips percentage'] = round(new_df['Sum'] / new_df[['Escooter Arrivals', 'Sum']].sum(axis=1) * 100, 2)
new_df['Rank'] = new_df['Sum'].rank(method='min')
new_df['Rank p'] = new_df['Lost trips percentage'].rank(method='min')

new_column_order = ['Filename', 'Escooter Arrivals', 'Escooter Starvations', 'Battery Starvations', 'Battery Violations', 'Sum', 'Lost trips percentage', 'Rank', 'Rank p']
new_df = new_df[new_column_order]
new_df = new_df.sort_values('Filename')

new_df

,Filename,Escooter Arrivals,Escooter Starvations,Battery Starvations,Battery Violations,Sum,Lost trips percentage,Rank,Rank p
24,criticality_weights_a_Base.csv,26351.842105,13068.368421,1216.842105,0.0,14285.210526,35.15,4.0,4.0
4,criticality_weights_aa_Base.csv,26294.631579,13117.052632,1256.263158,0.0,14373.315789,35.34,9.0,9.0
17,criticality_weights_ab_Base.csv,26225.947368,13124.105263,1325.789474,0.0,14449.894737,35.52,21.0,21.0
19,criticality_weights_ac_Base.csv,26210.631579,13182.368421,1271.368421,0.0,14453.736842,35.54,24.0,23.0
29,criticality_weights_ad_Base.csv,26366.894737,12987.210526,1282.473684,0.0,14269.684211,35.12,3.0,3.0
12,criticality_weights_b_Base.csv,26429.263158,13073.789474,1177.315789,0.0,14251.105263,35.03,1.0,1.0
13,criticality_weights_c_Base.csv,26289.789474,13194.368421,1201.947368,0.0,14396.315789,35.38,13.0,11.0
3,criticality_weights_d_Base.csv,26400.578947,13132.157895,1135.263158,0.0,14267.421053,35.08,2.0,2.0
2,criticality_weights_e_Base.csv,26187.105263,13145.631579,1305.315789,0.0,14450.947368,35.56,23.0,24.0
20,criticality_weights_f_Base.csv,26125.947368,13319.947368,1202.421053,0.0,14522.368421,35.73,29.0,30.0
